# Monge-Ampere Equation Multigrid Solver

In [0]:
import numpy 
%matplotlib inline
#from __future__ import print_function
import matplotlib.pyplot as plt
import scipy
from scipy import sparse
from scipy.sparse import spdiags
import scipy.sparse.linalg as linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

The plan is to do the following: 

- Explicit finite difference method with discretizing the second derivatives using standard central differences on a uniform Cartesian grid
$$(D^2_{xx}u_{i,j})(D^2_{yy}u_{i,j}) - (D^2_{xy}u_{i,j})^2 = f_{i,j}$$
- Discussion of monotonicity and exploring the alternate form of the MA equation
$$\det(D^2u) = u_{x_1x_1}u_{x_2x_2} - u_{x_1x_2}^2 \approx u_{v_1v_1}u_{v_2v_2} $$
where $v_1$ and $v_2$ are basis that give the minimum product of $u_{x_1x_1}u_{x_2x_2}$ considering all $x_1$ and $x_2$ 
- Implement full multigrid 
- Use different sequences of V-cycles


In [0]:
#A_solver.m file 

def A_solver(u, h, basesNum):
    """
    The function calculates u_{x_1,x_1}*u_{x_2,x_2} 
    
    :Input:
     - *u* (numpy.ndarray) 
     - *h* (float) gridsize 
     - *basesNum* (int) choice of basis use
    
    :Output:
     - *U* (numpy.darray)
    
    """
    
    # Calculate u_xx*u_yy, and leave only the nonnegative elements
    u_xy = numpy.maximum(1 / h**2 * ( u[0:-2,1:-1] + u[2:,1:-1] - 2 * u[1:-1,1:-1] ),0) \
         * numpy.maximum(1 / h**2 * ( u[1:-1,0:-2] + u[1:-1,2:] - 2 * u[1:-1,1:-1]), 0)
    
    if basesNum == 2:
        # Calculate u_vv*u_v(perp)v(perp), and leave only the nonnegative elements.
        u_vw = numpy.maximum(1/(2*h**2) * (u[2:,0:-2]+u[0:-2,2:]-2.*u[1:-1,1:-1]) , 0) \
               * numpy.maximum(1/(2*h**2) * (u[2:,2:]+u[0:-2,0:-2]-2.*u[1:-1,1:-1]) , 0)
        # Take the minimum of the two matrices.
        U = numpy.minimum(u_xy ,u_vw)
    
    else:
        U = u_xy
    
    return U

In [0]:
# init.m file
def init(F, g, n, h ,X, Y):
    """
    Calculates a reasonable initial guess to plug into the FAS function
    
    :Input:
     -*F*
     -*g*
     -*n*
     -*h*
     -*X*
     -*Y*
    
    :Output:
     -*u* (ndarray) initial guess with shape
    
    """
    m = n - 2
    
    F = numpy.sqrt(2 * F[1:-1, 1:-1])
    
    G = g(X, Y)
#     print(F.shape, G.shape, m, n)
    F[:,0] = F[:,0] - G[1:m,0] / h**2
    F[:, -1] = F[:, -1] - G[1:m, -1] / h**2
    F[0,:] = F[0,:] - G[0,1:m] / h**2
    F[-1,:] = F[-1,:] - G[-1,1:m] / h**2
    
    F = numpy.reshape(F,(m-1,m-1))
    
    e = numpy.ones(m-1)
    T = sparse.spdiags([e, -4.0 * e, e], [-1, 0, 1], m-1, m-1)
    S = sparse.spdiags([e, e], [-1, 1], m-1, m-1)
    I = sparse.eye(m-1)
    A = sparse.kron(I, T) + sparse.kron(S, I)
    A /= h**2
    
#     print(A.shape, F.shape)
    uVec = linalg.spsolve(A, F.reshape((m-1)**2, order='F'))
    G[1:-1,1:-1] = numpy.reshape(uVec,(m-1,m-1), order='F') #C-like index ordering
    u = G;

    return u

In [0]:
def looper2(F,g,n,N,levels,iterVec,h,u0,xa,xb,ya,yb,tol,mex):
    """
    ???
    :Input:
     -
    """
    count = 0
    u = u0
    res = 1
    resLast = 1e2
    
    while (res > tol) and (count < 15):
        count += 1
        u, resMat, err = FAS_V2(F, g, n, N, levels, iterVec, h, u, xa, xb, ya, yb, count, mex)
        res = numpy.linalg.norm(resMat, ord = numpy.inf)
        resLast = res
    
    return u,resMat,err,time,count

In [0]:
def FAS_V2(F,g,n,N,levels,iterVec,h,u0,xa,xb,ya,yb,count,mex):
    """
    FAS implements the Full Approximation Scheme
    :Inputs: 
     -*F*
     -*g*
     -*n*
     -*N*
     -*levels*
     -*iterVec*
     
     
    """
    direction = 'down'
    plotFigs = 0
    
    # Do the initial Gauss-Seidel iteration
    u, resMat, err = GaussSeidel(F,g,iterVec,h,u0,xa,xb,ya,yb,0,mex);
    res = numpy.linalg.norm(resMat, ord = numpy.inf)

    if plotFigs == 1:
        fig = plt.figure()
        axes = fig.add_subplot(111,projection='3d')
        Axes3D.scatter()
        # unfinished
        
    #Go down one level of granularity, setting a new n and h.
    n = (n + 1) / 2 
    h = 2*h 

    #Set the coarse versions of u, F, and the residual matrix.
    v = restrict(u)
    resCoarse = restrict(resMat)
    
    # Evaluate the MA equation on the coarse grid, pad it with zeros so that
    # it's the same size as resCoarse, and add them together.
    A = numpy.pad(A_solver(v,h,2),1, pad_with, padder=0) + resCoarse

    print('n = ',n)
    
    # If we're on the lowest level, run the Gauss-Seidel calculation and set
    # vNew as the output. Otherwise, set vNew as the recursive output of FAS_V.
    if numpy.floor(numpy.log2(N)) - numpy.floor(numpy.log2(n)) != levels: 
        vNew = FAS_V2(A,g,n,N,levels,iterVec,h,v,xa,xb,ya,yb,count,mex)
    
    else:
        vNew, resMat, err = GaussSeidel(A,g,iterVec,h,v,xa,xb,ya,yb,1,mex)
    
    direction = 'up'
    
    # Calculate the coarse error matrix.
    eCoarse = vNew - v

    # Interpolate the coarse error matrix to the fine level.
    eFine = enhance(eCoarse)

    # Add the error matrix to the original matrix u.
    print(u.shape, eFine.shape)
    u[1:-1,1:-1] = u[1:-1,1:-1] + eFine[1:-1,1:-1]

    # Come back up to the fine level.
    n = 2 * n - 1
    h = h / 2

    #Do a few more Gauss-Seidel iterations on u and calculate the residual matrix.
    u, resMat, err = GaussSeidel(F,g,iterVec,h,u,xa,xb,ya,yb,0,mex)
    res = numpy.linalg.norm(resMat, ord = numpy.inf)

    print('n = ',n)
    
    if plotFigs == 1:
        fig = plt.figure()
        axes = fig.add_subplot(111,projection='3d')
        Axes3D.scatter(resMat, 'linestyle')
        axes.title('n = %d, res = %f, count = %d',n,res,count)
        plt.show()
    
    # If we're on the finest level, calculate the error. Otherwise, set it to
    # zero so as to avoid a 'not enough output arguments' error.
    if n == N:
        x = numpy.arange(xa, xb, h)
        y = numpy.arange(ya, yb, h)
        [X,Y] = meshgrid(x,y)
        G = g(X,Y)
        err = G - u
    else:
        err = 0;
    
    return u, resMat, err

In [0]:
def enhance(u):
    """
    """
    n = 2 * len(u) - 1
    uFine = numpy.zeros((n,n))
    
    uFine[0::2,0::2] = u
    uFine[1:-1:2,0::2] = 0.5 * (u[0:-1,:] + u[1:,:])
    uFine[0::2,1:-1:2] = 0.5 * (u[:,0:-1] + u[:,1:])
    uFine[1:-1:2,1:-1:2] = 0.25 * (u[0:-1,0:-1] + u[1:,0:-1] + u[0:-1,1:] + u[1:,1:])
    
    return uFine

In [0]:
def restrict(u):
    """
    """
    U = u  

    # Pad u with zeros to make the calculation simpler.
    u = numpy.pad(u,1, pad_with, padder=0)

    # Set uCoarse according to a linear combination of points from u.
    uCoarse = 0.5 * u[1:-1:2,1:-1:2] + 0.125*(u[0:-2:2,1:-1:2] \
                                            + u[2::2,1:-1:2] \
                                            + u[1:-1:2,0:-2:2] \
                                            + u[1:-1:2,2::2])

    # Restrict u the old way by siply deleting every other point.
    u = U[0::2,0::2]

    # Reset the boundary values of uCoase with the correct boundary values.
    uCoarse[:,0] = u[:,0]
    uCoarse[:,-1] = u[:,-1]
    uCoarse[0,:] = u[0,:]
    uCoarse[-1,:] = u[-1,:]
    
    return uCoarse

In [0]:
def pad_with(vector, pad_width, iaxis, kwargs):
     pad_value = kwargs.get('padder', 10)
     vector[:pad_width[0]] = pad_value
     vector[-pad_width[1]:] = pad_value
     return vector

def GaussSeidel(F,g,iterVec,h,u0,xa,xb,ya,yb,coarse,mex):
    """
    
    """
    # Lambda is a constant used for calculating a weighted combination of the
    # existing u and the updated u.
    Lambda = 0.05

    # Set the grid on which the exact solution g(x,y) will be applied.
    x = numpy.arange(xa, xb, h)
    y = numpy.arange(ya, yb, h) 
    [X,Y] = numpy.meshgrid(x,y)
    G = g(X,Y)

    # Initialize the matrix u.
    u = u0

    # Set the right-hand side of the MA equation and the number of iterations 
    # in the loop depending on whether the input matrix u is on the coarse or 
    # the fine level.
    if coarse == 1:
        maxCount = iterVec[0]
    else:
        maxCount = iterVec[1]

    for cuenta in range(1, maxCount+1):
        if mex == 0:
            uNew = notJacobi(u, F, h, 2)
        else:
            m, n = size(u)
            uNew, hInv = notNotGaussSeidel(u,F,h,m,n)
            uNew = uNew[1:-1,1:-1]

        # This is the MATLAB code
#          figure(18)
#          u_Mat = notJacobi(u,F,h,2);

#          [m,n] = size(u);
#          u_C = notNotGaussSeidel(u,F,h,m,n);
#          u_C = u_C(2:end-1,2:end-1);

#          surf(abs(u_Mat - u_C))
#          surf(u_C)

#          uNew = notJacobi(u,F,h,2);

        # Update u with a weighted sum of points from the already-extant u and
        # the newly calculated uNew.
        u[1:-1,1:-1] = Lambda * u[1:-1,1:-1] + (1 - Lambda) * uNew

        resMat = numpy.pad(F[1:-1,1:-1] - A_solver(u,h,2), 1, pad_with, padder=0)

#         figure(18)
#         surf(resMat)
#         drawnow

    # Subtract u from the exact solution to find the error matrix.
    err = G - u

    #Pad the residual matrix with zeros to make it the right size again.
    resMat = numpy.pad(F[1:-1,1:-1] - A_solver(u,h,2),1, pad_with, padder=0)

    return u, resMat, err



In [0]:
def notJacobi(u,F,h,basesNum):
    """
    
    """
    for i in numpy.arange(1, u.shape[0] - 1):
        for j in numpy.arange(1, u.shape[1]-1):
            if basesNum == 2:
                A_xy = 1 / h**4 * ( u[i-1,j] + u[i+1,j] - 2 * u[i,j]) \
                              * (u[i,j-1]+u[i,j+1]-2*u[i,j])
                A_vw = 1/(4*h**4) * (u[i-1,j-1]+u[i+1,j+1] - 2 * u[i,j]) \
                                * (u[i+1,j-1] + u[i-1,j+1] - 2 * u[i,j])
            if A_xy <= A_vw: 
                u[i,j] = 0.25 * (u[i+1,j] + u[i-1,j] + u[i,j-1] + u[i,j+1]) \
                    -0.5 * numpy.sqrt(0.25*((u[i+1,j] + u[i-1,j] - u[i,j-1]-u[i,j+1]) ** 2) \
                    +h**4*F[i,j])
            else:
                u[i,j] = 0.25*(u[i-1,j+1]+u[i+1,j-1]+u[i+1,j+1]+u[i-1,j-1]) \
                    -0.5 * numpy.sqrt(0.25*((u[i-1,j+1] + u[i+1,j-1] - u[i+1,j+1] - u[i-1,j-1]) ** 2) \
                    +4*h**4 * F[i,j])
        else:
            u[i,j] = 0.25 * (u[i+1,j]+u[i-1,j]+u[i,j-1]+u[i,j+1]) \
                    -0.5 * numpy.sqrt(0.25*((u[i+1,j]+u[i-1,j]-u[i,j-1]-u[i,j+1])**2) \
                    +h**4 * F[i,j])
    u = u[1:-1,1:-1]
    
    return u


In [14]:
#main.m file

f1 = lambda x, y: (1 + x ** 2)*(1 + y**2)*numpy.exp(x**2 + y**2)
f2 = lambda x, y: numpy.exp(x**2 + y**2)*(1+0.5 * (x + y)**2)*(1 + 0.5 * (y - x)**2)
g = lambda x, y: numpy.exp((x**2 + y**2)/2)

# Set the minimum and maximum values of n for the solution matrix.
minN = 7;
maxN = 7;
stats = numpy.zeros((maxN - minN + 1, maxN + 1, 3))
stats[:,1,:] = 1

basesNum = 2 
iterVec = numpy.array([5, 50])

# k == 0 if we don't want to use the mex file, k == 1 if we do

for k in range(1):
    for i in range(minN, maxN + 1):
        print('i =',i)
        
        n = 2 **(i + 1) + 1
        h = 1 / (n - 1)
        xa = 0; xb = 1; ya = 0; yb = 1; tol = h**2 / 10
        x = numpy.arange(xa, xb, h)
        y = numpy.arange(ya, yb, h)
        [X, Y] = numpy.meshgrid(x, y)
        
        if basesNum == 1:
            F = f1(X,Y)
        else:
            F1 = f1(X, Y)
            F2 = f2(X, Y)
            F = numpy.minimum(F1, F2)
        
        G = g(X, Y)
        A = numpy.zeros((n, n))
        u0 = init(F, g, n, h, X, Y)
        
        u0[:,0] = g(X[:,0],Y[:,0])
        u0[:,-1] = g(X[:,-1],Y[:,-1])
        u0[0,:] = g(X[0,:],Y[0,:])
        u0[-1,:] = g(X[-1,:],Y[-1,:])
        
        N = n
        
        for j in range(1,i+1):
            u,resMat,error,time,count = looper2(F,g,n,N,j,2*iterVec,h,u0,xa,xb,ya,yb,tol,k)
            stats[i,j+1,k+1] = count
            save('stats.mat','stats')
        
            

i = 7
n =  129.0
(256, 256) (255, 255)


ValueError: ignored

In [3]:
import numpy
x = numpy.zeros((4,4))
len(x[:,0])

4